# IN PROGRESS ...

In [153]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime

In [159]:
url_main = 'https://www.foxsports.com/soccer/2022-fifa-world-cup/teams'
response = requests.get(url_main)
soup = BeautifulSoup(response.text, 'html.parser')

In [160]:
url_main.split('/')[-2]

'2022-fifa-world-cup'

In [145]:
country_name = [x.text for x in soup.find_all('h3')]
countries = [x['href'] for x in soup.find_all('a',{'class':'entity-list-row-container image-logo'})]

In [152]:
# running time: 10s
rows=[]
for idx,country in enumerate(countries):

    url = f'https://www.foxsports.com{x}-roster'
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    table = soup.find('div',{'view':'team'})
    title = [x.text.strip() for x in table.find_all('tr')[0]]
    
    for group in table.find_all('tbody')[:-1]:
        for player in group:
            row={}
            row['Country'] = country_name[idx].capitalize()
            row['Name'] = player.find('h3').text
            row[title[1]] = player.find('td',{'data-index':'1'}).text.strip()
            row[title[2]] = player.find('td',{'data-index':'2'}).text.strip()
            row[title[3]] = player.find('td',{'data-index':'3'}).text.strip()
            row[title[4]] = player.find('td',{'data-index':'4'}).text.strip()
            rows.append(row)

In [166]:
data = pd.DataFrame(rows)

date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = f"../ESPN - Output/{url_main.split('/')[-2]} {date} - Webscrape.csv"

data.to_csv(filePath, index=False) # save to file path

In [179]:
# Clean table

# TO DO:
# drop duplicated players within a country (ex: Argentina 'Agustin Rossi')

# build dataframe
data = pd.DataFrame(rows)

# drop rows with missing data
data = data[~data.isin(['-']).any(axis=1)]

# replace possition values
POS_values = {x.text.strip().capitalize() for x in table.find_all('th',{'data-index':'0'})}
POS_keys = set(data[title[1]].to_list())
POS_mapped = dict(zip(POS_keys, POS_values))
data.replace({title[1]: POS_mapped}, inplace=True)

# age column to number
data[title[2]] = pd.to_numeric(data[title[2]])

# hight column to number
data[title[3]] = data[title[3]].apply(lambda x: (int(x.split('\'')[0])*12 + int(x.split('\'')[1].replace('\"',''))) * 2.54) # inches to centimeters

# weight column to number
data[title[4]] = data[title[4]].apply(lambda x: round(int(x.split(' ')[0]) / 2.205,1)) # lbs to kg

data


,Country,Name,POS,AGE,HT,WT
0,Argentina,Song Beom-keun,Goalkeeper,24,193.04,91.6
1,Argentina,Sung-yun Gu,Goalkeeper,28,193.04,81.6
2,Argentina,Jo Hyeon-uh,Goalkeeper,30,187.96,74.8
4,Argentina,Jin Hyeon Kim,Goalkeeper,35,190.50,81.6
5,Argentina,Kim Dong-jun,Goalkeeper,27,187.96,73.9
...,...,...,...,...,...,...
2168,Wales,Shin-wook Kim,Player,34,195.58,96.6
2171,Wales,Song Min-kyu,Player,22,177.80,71.7
2172,Wales,Jeong Sang-Bin,Player,20,172.72,67.6
2173,Wales,Eom Won-sang,Player,23,170.18,62.6


In [151]:
# Average per possition
data.loc[:, data.columns != data.columns[0]].groupby('POS').mean().round(1)

,AGE,HT,WT
POS,,,
Defender,29.1,179.5,71.9
Goalkeeper,29.2,189.7,81.2
Midfielder,27.2,176.2,69.4
Player,26.2,179.7,74.5


In [47]:
# Average per possition
data.loc[:, data.columns != data.columns[0]].groupby('POS').mean().round(1)

,AGE,HT,WT
POS,,,
Defender,26.9,180.2,75.8
Goalkeeper,30.2,189.5,87.0
Midfielder,26.9,174.8,70.3
Player,26.5,177.6,75.7
